In [10]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Configuration
IMG_SIZE = 160
base_dir = "images"
occlusion_types = ["masked", "sunglasses", "neutral"]

print("Counting total images...")
total_images = sum(
    len(os.listdir(os.path.join(base_dir, occlusion_type, person_dir)))
    for occlusion_type in occlusion_types if os.path.exists(os.path.join(base_dir, occlusion_type))
    for person_dir in os.listdir(os.path.join(base_dir, occlusion_type))
    if os.path.isdir(os.path.join(base_dir, occlusion_type, person_dir))
)

print(f"Total images: {total_images}")

# Containers for data
images, labels = [], []
label_names = {}

# Function to process a single image
def process_image(img_path, label):
    try:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Load as color (BGR)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize image
        img = img.astype("float32") / 255.0  # Normalize
        return img, label
    except Exception as e:
        print(f"[ERROR] Failed to load image {img_path}: {e}")
        return None, None

# Multi-threaded image loading
def load_images():
    tasks = []
    with ThreadPoolExecutor(max_workers=8) as executor:  # Adjust number of workers as needed
        for occlusion_type in occlusion_types:
            occlusion_path = os.path.join(base_dir, occlusion_type)

            if not os.path.exists(occlusion_path):
                print(f"[WARNING] Directory {occlusion_path} does not exist. Skipping.")
                continue

            for person_dir in os.listdir(occlusion_path):
                person_path = os.path.join(occlusion_path, person_dir)

                if not os.path.isdir(person_path):
                    continue

                if person_dir not in label_names:
                    label_names[person_dir] = len(label_names)  # Assign numeric label
                
                label = label_names[person_dir]

                for img_file in os.listdir(person_path):
                    if img_file.endswith('.jpg'):
                        img_path = os.path.join(person_path, img_file)
                        tasks.append(executor.submit(process_image, img_path, label))

        for future in tqdm(tasks, total=total_images, desc="Processing images", ncols=100):
            img, label = future.result()
            if img is not None:
                images.append(img)
                labels.append(label)

# Start loading images
load_images()

# Convert to NumPy arrays
images = np.array(images, dtype="float32")
labels = np.array(labels, dtype="int")

print(f"Dataset Loaded: {images.shape[0]} images, {len(set(labels))} unique labels.")


Counting total images...
Total images: 7827
[ERROR] Failed to load image images\masked\anthony_davis_wearing_mask\000001.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\masked\amy_klobuchar_wearing_mask\000004.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\masked\bill_gates_wearing_mask\000002.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\masked\cory_booker_wearing_mask\000004.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in 

Processing images:   0%|                                                   | 0/7827 [00:00<?, ?it/s]

[ERROR] Failed to load image images\sunglasses\jack_nicholson_wearing_sunglasses\000026.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\sunglasses\james_harden_wearing_sunglasses\000025.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



Processing images:  20%|███████▌                             | 1588/7827 [00:00<00:00, 13815.54it/s]

[ERROR] Failed to load image images\sunglasses\jeff_bridges_wearing_sunglasses\000013.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\sunglasses\joaquin_phoenix_wearing_sunglasses\000012.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\sunglasses\jon_favreau_wearing_sunglasses\000007.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\sunglasses\kevin_spacey_wearing_sunglasses\000020.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in f

...

[ERROR] Failed to load image images\neutral\michelle_pfeiffer\000016.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\neutral\neymar\000027.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\neutral\novak_djokovic\000018.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\neutral\patty_murray\000029.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



Processing images:  97%|████████████████████████████████████▉ | 7605/7827 [00:02<00:00, 2095.30it/s]

[ERROR] Failed to load image images\neutral\steve_buscemi\000027.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

[ERROR] Failed to load image images\neutral\tammy_baldwin\000019.jpg: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



Processing images: 100%|██████████████████████████████████████| 7827/7827 [00:02<00:00, 3330.16it/s]


Dataset Loaded: 7781 images, 401 unique labels.


In [11]:
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np

# Step 1: Count instances per class
class_counts = Counter(labels)

# Step 2: Remove classes with fewer than 3 images (to allow train/test split)
valid_classes = {cls for cls, count in class_counts.items() if count >= 3}
filtered_images, filtered_labels = [], []

for img, lbl in zip(images, labels):
    if lbl in valid_classes:
        filtered_images.append(img)
        filtered_labels.append(lbl)

images = np.array(filtered_images, dtype="float32")
labels = np.array(filtered_labels, dtype="int")

print(f"Filtered dataset: {images.shape[0]} images, {len(set(labels))} unique classes")

# Step 3: First split (Train & Temp) - Stratified
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.3, random_state=42, stratify=labels  # 70% Train, 30% Temp
)

# Step 4: Check and remove single-instance classes from `y_temp`
temp_counts = Counter(y_temp)
valid_temp_classes = {cls for cls, count in temp_counts.items() if count > 1}

X_temp_filtered, y_temp_filtered = [], []
for img, lbl in zip(X_temp, y_temp):
    if lbl in valid_temp_classes:
        X_temp_filtered.append(img)
        y_temp_filtered.append(lbl)

X_temp_filtered = np.array(X_temp_filtered, dtype="float32")
y_temp_filtered = np.array(y_temp_filtered, dtype="int")

# Step 5: Second split (Validation & Test) - No stratification to avoid errors
X_val, X_test, y_val, y_test = train_test_split(
    X_temp_filtered, y_temp_filtered, test_size=0.5, random_state=42
)

# Print results
print(f"Training set: {X_train.shape[0]} images")
print(f"Validation set: {X_val.shape[0]} images")
print(f"Test set: {X_test.shape[0]} images")


Filtered dataset: 7714 images, 356 unique classes
Training set: 5399 images
Validation set: 1139 images
Test set: 1139 images


In [12]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Configuration parameters
IMG_SIZE = 160
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
num_classes = len(set(labels))  # number of unique identities

# Build the model using MobileNetV2 as the base
input_shape = (IMG_SIZE, IMG_SIZE, 3)
base_model = MobileNetV2(include_top=False, input_shape=input_shape, weights='imagenet')
base_model.trainable = False  # Freeze the base model for now

inputs = Input(shape=input_shape)
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
model.summary()

# Define a simple early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop]
)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print(f"Test Accuracy: {test_acc:.4f}")

# (Optional) If you'd like to see predictions on a few test images:
import matplotlib.pyplot as plt
import numpy as np

# Plot a few sample predictions
num_samples = 5
indices = np.random.choice(len(X_test), num_samples, replace=False)
predictions = model.predict(X_test[indices])
predicted_classes = np.argmax(predictions, axis=1)

for i, idx in enumerate(indices):
    plt.figure()
    plt.imshow(X_test[idx])
    plt.title(f"True: {y_test[idx]}, Pred: {predicted_classes[i]}")
    plt.axis('off')
    plt.show()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 356)            │       456,036 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,714,020 (10.35 MB)

 Trainable params: 456,036 (1.74 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\grant\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 639, in run_forever

  File "C:\Users\grant\AppDat